In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from torchvision import datasets, transforms

from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.utils import common_functions as c_f
from pytorch_metric_learning.utils.inference import InferenceModel, MatchFinder
import features
import models
import torchvision

In [2]:
trained = False

In [3]:
def print_decision(is_match):
    if is_match:
        print("Same class")
    else:
        print("Different class")


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

inv_normalize = transforms.Normalize(
    mean=[-m / s for m, s in zip(mean, std)], std=[1 / s for s in std]
)


def imshow(img, figsize=(8, 4)):
    img = inv_normalize(img)
    npimg = img.numpy()
    plt.figure(figsize=figsize)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)]
)

In [5]:
dataset = features.PypyrusDataset(data='/Users/beantown/PycharmProjects/master-thesis/data/processed/06_dataset_3/val',
                               csv='/Users/beantown/PycharmProjects/master-thesis/data/processed/06_dataset_3/info.csv',
                               mode='val',
                               transform=transform,
                               debug=False,
                               batch_size=64)

In [6]:
labels_to_indices = c_f.get_labels_to_indices(dataset.targets)

In [11]:
trunk = torchvision.models.densenet121(pretrained=False)
trunk_output_size = trunk.classifier.in_features
trunk.classifier = c_f.Identity()
trunk.load_state_dict(torch.load("/Users/beantown/PycharmProjects/master-thesis/results/ms1/11_Dataset_3_Scheduler_Densenet121_MultiSimilarityLoss_MultiSimilarityMiner/saved_models/trunk_best13.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [12]:
embedder = models.MLP([trunk_output_size, 64]).to(torch.device('cpu'))
embedder.load_state_dict(torch.load("/Users/beantown/PycharmProjects/master-thesis/results/ms1/11_Dataset_3_Scheduler_Densenet121_MultiSimilarityLoss_MultiSimilarityMiner/saved_models/embedder_best13.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [18]:
classifier = models.MLP([64, 50]).to(torch.device('cpu'))
classifier.load_state_dict(torch.load("/Users/beantown/PycharmProjects/master-thesis/results/ms1/11_Dataset_3_Scheduler_Densenet121_MultiSimilarityLoss_MultiSimilarityMiner/saved_models/classifier_best13.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [19]:
# cars and frogs
classA, classB = labels_to_indices[13796], labels_to_indices[1402]

In [20]:
#labels_to_indices

In [31]:
import PIL

In [35]:
from skimage import io


#image = io.imread('/Users/beantown/PycharmProjects/master-thesis/data/processed/03_post_cleansing/0_13927_4814C4V.png')
image = PIL.Image.open('/Users/beantown/PycharmProjects/master-thesis/data/processed/03_post_cleansing/0_13927_4814C4V.png')
#io.imshow(image)

In [43]:
x = transform(image)

In [45]:
x.shape

torch.Size([3, 2557, 3544])

In [46]:
def batch_predict(images):
    
    transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
    x = transform(image)
    x = x[None, :]
    
    trunk.eval()
    embedder.eval()
    classifier.eval()

    
    x = trunk(x)
    x = embedder(x)
    logits = classifier(x)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

In [48]:
import torch.nn.functional as F

In [49]:
test_pred = batch_predict([pill_transf(image)])
test_pred.squeeze().argmax()

0

In [51]:
from lime import lime_image

In [55]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(x.T, 
                                         batch_predict, # classification function
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=1000) # number of images that will be sent to classification function

ValueError: Buffer dtype mismatch, expected 'double' but got 'float'